In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments
import torch
import os
import pandas as pd
from tqdm import tqdm
from datasets import Dataset
from fuzzywuzzy import fuzz
import evaluate
import csv
import numpy as np
import torch.nn.functional as F
from sklearn.metrics import top_k_accuracy_score, ndcg_score
import random

/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/goodreads/splits/train.jsonl"
df_recommender_train = pd.read_json(input_file, lines=True)
for _, row in df_recommender_train.iterrows():
    row["recommended_book"]["book_name"] = row["recommended_book"]["book_name"].lower()

In [4]:
df_recommender_train

,user_id,user_previous_interactions,recommended_book,negative_recommended_book,turns
0,8842281e1d1347389f2ab93d60773d4d,[{'book_name': 'Web Standards Solutions: The M...,"{'book_name': 'pride and prejudice', 'book_id'...","[{'book_name': 'the fiery cross (outlander, #5...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1,8842281e1d1347389f2ab93d60773d4d,[{'book_name': 'Web Standards Solutions: The M...,{'book_name': 'the tipping point: how little t...,"[{'book_name': 'night (the night trilogy #1)',...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
2,8842281e1d1347389f2ab93d60773d4d,[{'book_name': 'Web Standards Solutions: The M...,"{'book_name': 'surely you're joking, mr. feynm...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
3,0ccc92b2cef99f22af2b28307400dd19,[],"{'book_name': 'the great gatsby', 'book_id': '...","[{'book_name': 'to kill a mockingbird', 'book_...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
4,5b1a46a337306e95d09bf3d1a7d2289e,[{'book_name': 'The Valley of Horses (Earth's ...,"{'book_name': 'the secret life of bees', 'book...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
...,...,...,...,...,...
8998,e55909b0cc4b0ebf426b7e49185ab502,"[{'book_name': 'Eve (Eve, #1)', 'book_id': '92...","{'book_name': 'a monster calls', 'book_id': '8...",[{'book_name': 'city of heavenly fire (the mor...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
8999,3f8351b28cb6060aba4872c3c783fa01,"[{'book_name': 'Bully (Fall Away, #1)', 'book_...","{'book_name': 'mr. president (white house, #1)...","[{'book_name': 'real (real, #1)', 'book_id': '...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
9000,b70962b73d3a34b919b3da0cfd71cd13,[{'book_name': 'Kitty Steals the Show (Kitty N...,"{'book_name': 'pet sematary', 'book_id': '2336...","[{'book_name': 'bag of bones', 'book_id': '105...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
9001,2c8d4f471c943c1f13e34a42bd8034b3,"[{'book_name': 'Every Heart', 'book_id': '2365...","{'book_name': 'elicit', 'book_id': '31737890',...","[{'book_name': 'the deep end (honey, #1)', 'bo...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."


In [5]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/goodreads/splits/val.jsonl"
df_recommender_validation = pd.read_json(input_file, lines=True)
for _, row in df_recommender_validation.iterrows():
    row["recommended_book"]["book_name"] = row["recommended_book"]["book_name"].lower()

In [6]:
df_recommender_validation

,user_id,user_previous_interactions,recommended_book,negative_recommended_book,turns
0,5a4dc04139e3341aab1646ff79128f25,[{'book_name': 'Apocalypse: The Lords of Deliv...,"{'book_name': 'marriage games (the games duet,...","[{'book_name': 'aced (driven, #4)', 'book_id':...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1,b7cb5ca2718789c2ea81b00a7e887073,"[{'book_name': 'Magic Breaks (Kate Daniels, #7...",{'book_name': 'gardens of the moon (the malaza...,"[{'book_name': 'stormwalker (stormwalker, #1)'...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
2,0c2511284f1a56239d879d5f29dc95a9,"[{'book_name': 'Darkfever (Fever, #1)', 'book_...","{'book_name': 'unsuitable', 'book_id': '321957...","[{'book_name': 'the ending i want', 'book_id':...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
3,5a4dc04139e3341aab1646ff79128f25,[{'book_name': 'Apocalypse: The Lords of Deliv...,{'book_name': 'american queen (new camelot tri...,"[{'book_name': 'chances (lucky santangelo, #1)...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
4,6c39913e49c45ca8682891ff8664faf9,"[{'book_name': 'Mary and O'Neil', 'book_id': '...","{'book_name': 'blister', 'book_id': '30373904'...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
...,...,...,...,...,...
1927,aa1ddc7ae913cdd7360335853ec8b99b,"[{'book_name': 'Let the Wind Speak', 'book_id'...","{'book_name': 'all the light we cannot see', '...","[{'book_name': 'the kite runner', 'book_id': '...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1928,6c39913e49c45ca8682891ff8664faf9,"[{'book_name': 'Mary and O'Neil', 'book_id': '...","{'book_name': 'almost missed you', 'book_id': ...",[{'book_name': 'the thing on the doorstep and ...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1929,58c8f055fc71eb77eb3260faa622edc2,[{'book_name': 'Banking the Billionaire (Billi...,"{'book_name': 'most valuable playboy', 'book_i...",[{'book_name': 'american queen (new camelot tr...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1930,b70962b73d3a34b919b3da0cfd71cd13,[{'book_name': 'Kitty Steals the Show (Kitty N...,"{'book_name': 'golden son', 'book_id': '254305...","[{'book_name': 'morning star (red rising, #3)'...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."


In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name_or_path = "google/flan-t5-base")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base", additional_special_tokens=["computer:", "human:"])
model.resize_token_embeddings(len(tokenizer))
IGNORE_INDEX = -100

/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
prompt_train = []
recommend_train = []
not_founds = 0

for _, row in tqdm(df_recommender_train.iterrows(), total=len(df_recommender_train)):
    prompt = ""
    found = False
    recommended = row["recommended_book"]["book_name"]
    
    for index, turn in enumerate(row["turns"]):
        if "COMPUTER" in turn:
            computer = turn["COMPUTER"]
        
            if fuzz.partial_ratio(recommended, computer.lower()) >= 95:
                prompt += "computer: I would recommend the "
                prompt_train.append(prompt)
                recommend_train.append(recommended)
                found = True
                break
            else:
                prompt += "computer: "+ computer + "\n"
        
        if "HUMAN" in turn:
            human = turn["HUMAN"]
            prompt += "human: " + human + "\n"
    
    if not found:
        not_founds += 1

print(f"Could not find {not_founds}")
print(f"len(prompt_train): {len(prompt_train)}")
print(f"len(recommend_train): {len(recommend_train)}")

            
            
prompt_encodings = tokenizer(prompt_train, padding='max_length', max_length=1024, truncation=True, return_tensors='pt')
recommend_encodings = tokenizer(recommend_train, padding='max_length', max_length=32, truncation=True, return_tensors='pt')

labels = recommend_encodings['input_ids']
labels[labels == tokenizer.pad_token_id] = IGNORE_INDEX

dataset = {
    'input_ids': prompt_encodings['input_ids'],
    'attention_mask': prompt_encodings['attention_mask'],
    'labels': labels,
}
dataset_train = Dataset.from_dict(dataset)

  0%|          | 0/9003 [00:00<?, ?it/s]

100%|██████████| 9003/9003 [00:42<00:00, 213.59it/s]


Could not find 35
len(prompt_train): 8968
len(recommend_train): 8968


In [9]:
prompt_validation = []
recommend_validation = []
not_founds = 0

for _, row in tqdm(df_recommender_validation.iterrows(), total=len(df_recommender_validation)):
    prompt = ""
    found = False
    recommended = row["recommended_book"]["book_name"]
    
    for index, turn in enumerate(row["turns"]):
        if "COMPUTER" in turn:
            computer = turn["COMPUTER"]
            
            if fuzz.partial_ratio(recommended, computer.lower()) >= 95:
                prompt += "computer: I would recommend the "
                prompt_validation.append(prompt)
                recommend_validation.append(recommended)
                found = True
                break
            else:
                prompt += "computer: "+ computer + "\n"
        
        if "HUMAN" in turn:
            human = turn["HUMAN"]
            prompt += "human: " + human + "\n"
    
    if not found:
        not_founds += 1
        
print(f"Could not find {not_founds}")
print(f"len(prompt_validation): {len(prompt_validation)}")
print(f"len(recommend_validation): {len(recommend_validation)}")
            
            
prompt_encodings = tokenizer(prompt_validation, padding='max_length', max_length=1024, truncation=True, return_tensors='pt')
recommend_encodings = tokenizer(recommend_validation, padding='max_length', max_length=32, truncation=True, return_tensors='pt')

labels = recommend_encodings['input_ids']
labels[labels == tokenizer.pad_token_id] = IGNORE_INDEX

dataset = {
    'input_ids': prompt_encodings['input_ids'],
    'attention_mask': prompt_encodings['attention_mask'],
    'labels': labels,
}
dataset_validation = Dataset.from_dict(dataset)

100%|██████████| 1932/1932 [00:08<00:00, 224.53it/s]


Could not find 5
len(prompt_validation): 1927
len(recommend_validation): 1927


In [10]:
def data_collator(batch):
    input_ids, attention_mask, labels,  = [], [], []
    for sample in batch:
        input_ids.append(sample['input_ids'])
        attention_mask.append(sample['attention_mask'])
        labels.append(sample['labels'])
    max_encoder_len = max(sum(x) for x in attention_mask)
    max_decoder_len = max(sum([0 if item == IGNORE_INDEX else 1 for item in x]) for x in labels)
    return {
        'input_ids': torch.tensor(input_ids)[:, :max_encoder_len],
        'attention_mask': torch.tensor(attention_mask)[:, :max_encoder_len],
        'labels': torch.tensor(labels)[:, :max_decoder_len]
    }

In [11]:
training_args = TrainingArguments(
    output_dir="/u/spa-d4/grad/mfe261/Projects/MobileConvRec/models/new_models/goodreads/T5_recommender",
    num_train_epochs=5,
    # logging_steps=500,
    # logging_dir=self.cfg.logging_dir,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_strategy="steps",
    evaluation_strategy="steps",
    save_steps=0.3,#self.cfg.save_steps,
    eval_steps=0.3, #self.cfg.eval_steps,
    save_total_limit=3,
    gradient_accumulation_steps=3, #gradient_accumulation_steps,
    per_device_train_batch_size=4, #train_batch_size,
    per_device_eval_batch_size=4, #self.cfg.eval_batch_size,
    warmup_steps=100,
    weight_decay=0.01,
    # dataloader_drop_last=True,
    disable_tqdm=False,
    push_to_hub=False
)

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset_train,
        eval_dataset=dataset_validation,
        data_collator=data_collator,
    )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
trainer.train() # resume_from_checkpoint=True
trainer.save_model()

Step,Training Loss,Validation Loss
1121,2.950400,3.065450
2242,2.666100,2.961706
3363,2.500900,2.932088


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


# Load the model and test it on the test dataset

In [13]:
torch.cuda.empty_cache()

In [14]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/goodreads/splits/test.jsonl"
df_recommender_test = pd.read_json(input_file, lines=True)
for _, row in df_recommender_test.iterrows():
    row["recommended_book"]["book_name"] = row["recommended_book"]["book_name"].lower()

In [15]:
apps_training_path = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/goodreads/goodreads_df.csv"

all_apps = []
with open(apps_training_path, 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        all_apps.append(row["title"].lower())
        
all_apps = list(set(all_apps))

In [16]:
def candidate_creator(row):
    np.random.seed(row.name)
    selected_values = np.random.choice(np.setdiff1d( all_apps, [row["recommended_book"]["book_name"]]), 24, replace=False) # filter_candidate_apps(row["recommended_book"]["book_name"])
    random_position = np.random.randint(0, len(selected_values) + 1)
    
    return np.insert(selected_values, random_position, row["recommended_book"]["book_name"]) 

df_recommender_test['candidate'] = df_recommender_test.apply(lambda row: candidate_creator(row), axis=1)

In [17]:
prompt_test = []
recommend_test = []
candidate_books = []
true_candidate_indexes = []
not_founds = 0
for _, row in df_recommender_test.iterrows():
    candidates = []
    for index, candidate_book in enumerate(row["candidate"].tolist()):
        candidates.append(candidate_book)
        if candidate_book == row["recommended_book"]["book_name"]:
            true_candidate_index = index
    prompt = ""
    
    found = False
    recommended = row["recommended_book"]["book_name"]
    
    for index, turn in enumerate(row["turns"]):
        computer = turn["COMPUTER"]
        
        if fuzz.partial_ratio(recommended, computer.lower()) >= 95:
            prompt += "computer: I would recommend the "
            prompt_test.append(prompt)
            recommend_test.append(recommended)
            candidate_books.append(candidates)
            true_candidate_indexes.append(true_candidate_index)
            found = True
            break
        else:
            prompt += "computer: "+ computer + "\n"
        
        if "HUMAN" in turn:
            human = turn["HUMAN"]
            prompt += "human: " + human + "\n"
    
    if not found:
        not_founds += 1

print(f"Could not find {not_founds}")
print(f"Number of prompt: {len(prompt_test)}")
print(f"Number of generations: {len(recommend_test)}")
print(f"Number of candidate apps: {len(candidate_books)}")
print(f"Number of true candidate indexes: {len(true_candidate_indexes)}")

Could not find 7
Number of prompt: 1916
Number of generations: 1916
Number of candidate apps: 1916
Number of true candidate indexes: 1916


In [18]:
model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name_or_path = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/models/new_models/goodreads/T5_recommender")
model.eval()
model = model.to('cuda')
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base", additional_special_tokens=["computer:", "human:"])
model.resize_token_embeddings(len(tokenizer))
IGNORE_INDEX = -100

/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [26]:
def chunk(list_of_elements, batch_size): # using this chunk function, we can split our data to multiple batches
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]

def evaluate_recommender(prompt_test, recommend_test, model, tokenizer, batch_size=8, threshold=70):
  prompt_batches = list(chunk(prompt_test, batch_size))
  generation_batches = list(chunk(recommend_test, batch_size))

  correctly_predicted = []
  for prompt_batch, generation_batch in tqdm(zip(prompt_batches, generation_batches), total = len(generation_batches)):

    inputs = tokenizer(prompt_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt") 

    generations_predicted = model.generate(input_ids=inputs["input_ids"].to('cuda'), attention_mask=inputs["attention_mask"].to('cuda'),
                            max_new_tokens=32,
                            num_beams=8,
                            eos_token_id=tokenizer.eos_token_id,
                            pad_token_id=tokenizer.pad_token_id,
                            bos_token_id=tokenizer.bos_token_id) # length_penalty=0.8, Set length_penalty to values < 1.0 in order to encourage the model to generate shorter sequences, to a value > 1.0 in order to encourage the model to produce longer sequences.

    decoded_generations = [tokenizer.decode(generation, skip_special_tokens=True, clean_up_tokenization_spaces=True) for generation in generations_predicted]
    generation_batch = [generation for generation in generation_batch]
    
    correctly_predicted.extend([1 if fuzz.ratio(predicted, ground_truth) > threshold else 0 for predicted, ground_truth in zip(decoded_generations, generation_batch)])

  return correctly_predicted

In [20]:
correctly_predicted = evaluate_recommender(prompt_test, recommend_test, model, tokenizer, batch_size=8, threshold=70)
success_rate = sum(correctly_predicted) / len(correctly_predicted)
print("success_rate: ", success_rate)

  0%|          | 0/240 [00:00<?, ?it/s]

100%|██████████| 240/240 [04:45<00:00,  1.19s/it]

success_rate:  0.009916492693110648


In [21]:
def chunk(list_of_elements, batch_size): # using this chunk function, we can split our data to multiple batches
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]
    
def convert_to_sublists(numbers, sublist_size):
    return [numbers[i:i+sublist_size] for i in range(0, len(numbers), sublist_size)]

def recommender_rank(prompts, candidate_apps, model, tokenizer, batch_size=8):
  model.eval()
  encoder_max_length = 1024
  decoder_max_length = 32
  prompts_tokenized = tokenizer(prompts, max_length=encoder_max_length, truncation=True, padding="max_length", return_tensors="pt")
  
  input_ids_decoder = []
  attention_mask_decoder = []
  input_ids_encoder = []
  attention_mask_encoder  = []
  for index, candidate_app_elements in enumerate(candidate_apps):
    candidate_app_elements = [tokenizer.pad_token+element for element in candidate_app_elements] # adding pad token to the beginning of each candidate app
    candidate_apps_tokenized = tokenizer(candidate_app_elements, max_length=decoder_max_length, truncation=True, padding="max_length", return_tensors="pt")
    for candidate_app_index in range(len(candidate_app_elements)):
      input_ids_decoder.append(candidate_apps_tokenized["input_ids"][candidate_app_index])
      attention_mask_decoder.append(candidate_apps_tokenized["attention_mask"][candidate_app_index])
      input_ids_encoder.append(prompts_tokenized["input_ids"][index])
      attention_mask_encoder.append(prompts_tokenized["attention_mask"][index])
  
  input_ids_encoder_batches = list(chunk(input_ids_encoder, batch_size))
  attention_mask_encoder_batches = list(chunk(attention_mask_encoder, batch_size))
  input_ids_decoder_batches = list(chunk(input_ids_decoder, batch_size))
  attention_mask_decoder_batches = list(chunk(attention_mask_decoder, batch_size))
  

  scores = []
  for input_ids_encoder_batch, attention_mask_encoder_batch, input_ids_decoder_batch, attention_mask_decoder_batch in tqdm(zip(input_ids_encoder_batches, attention_mask_encoder_batches, input_ids_decoder_batches, attention_mask_decoder_batches), total = len(input_ids_encoder_batches)):
    decoder_input_ids = torch.stack(input_ids_decoder_batch).to("cuda")
    decoder_attention_mask = torch.stack(attention_mask_decoder_batch).to("cuda")
    input_ids = torch.stack(input_ids_encoder_batch).to("cuda")
    attention_mask = torch.stack(attention_mask_encoder_batch).to("cuda")
    with torch.no_grad():
      model_output = model(decoder_input_ids=decoder_input_ids, decoder_attention_mask=decoder_attention_mask, 
                           input_ids=input_ids, attention_mask=attention_mask)
    
    logprobs = F.log_softmax(model_output["logits"], dim=-1)[:, :-1, :] # remove the eos token
    output_tokens = decoder_input_ids[:, 1:] # remove the bos token
        
    tokens_logprobs = torch.gather(logprobs, 2, output_tokens[:, :, None]).squeeze(-1).to(torch.float32)
        
    mask = torch.ones(tokens_logprobs.shape, dtype=torch.bool, device="cuda")
    for i, _output in enumerate(output_tokens):
      for j, _token in enumerate(_output):
        if _token == tokenizer.pad_token_id:
          mask[i, j] = False
              
    score = (tokens_logprobs * mask).sum(-1) / mask.sum(-1)
    scores.extend(score.to('cpu').tolist())
    
  # batch_input_representations = torch.cat(batch_input_representations)
  
  scores = convert_to_sublists(scores, len(candidate_apps[0]))
                
  return scores

In [22]:
scores = recommender_rank(prompt_test, candidate_books, model, tokenizer, batch_size=8)

100%|██████████| 5988/5988 [22:33<00:00,  4.42it/s]


Sampled Candidates

In [23]:
# that is the one
[top_k_accuracy_score(true_candidate_indexes, scores, k=k) for k in range(1, 11)]

[np.float64(0.07985386221294363),
 np.float64(0.1289144050104384),
 np.float64(0.16805845511482254),
 np.float64(0.2150313152400835),
 np.float64(0.2588726513569937),
 np.float64(0.2980167014613779),
 np.float64(0.3376826722338205),
 np.float64(0.3695198329853862),
 np.float64(0.40970772442588727),
 np.float64(0.4384133611691023)]

In [24]:
true_relevance = [[1 if item == index else 0 for item in range(len(candidate_books[0]))] for index in true_candidate_indexes]

In [25]:
# that is the one
[ndcg_score(true_relevance, scores, k=k) for k in range(1, 11)]

[np.float64(0.07985386221294363),
 np.float64(0.11080761839024895),
 np.float64(0.13037964344244102),
 np.float64(0.1506097531640513),
 np.float64(0.16756989711379114),
 np.float64(0.18151328909348932),
 np.float64(0.19473527935097018),
 np.float64(0.20477878534467034),
 np.float64(0.21687654613074886),
 np.float64(0.22517433603027073)]

Similar Candidates

In [ ]:
[top_k_accuracy_score(true_candidate_indexes, scores, k=k) for k in range(1, 11)]

In [44]:
true_relevance = [[1 if item == index else 0 for item in range(len(candidate_books[0]))] for index in true_candidate_indexes]

In [ ]:
[ndcg_score(true_relevance, scores, k=k) for k in range(1, 11)]